In [17]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from dotenv import load_dotenv

In [3]:
#  Fetches Company name, sector, industry, long business summary, current market price.
def get_company_info(ticker_s):
    ticker = yf.Ticker(ticker_s)
    information = ticker.info
    name = information.get('longName')
    sector = information.get('sector')
    industry = information.get('industry')
    summary = information.get('longBusinessSummary')
    hist = ticker.history(period="5d")
    current_price = hist['Close'].iloc[-1]


    company_info = (
            f"Company Name: {name}\n"
            f"Sector: {sector}\n"
            f"Industry: {industry}\n"
            f"Current Market Price: {current_price:.2f}\n"
            f"Business Summary: {summary}"
        )
    return company_info


#Fetches P/E ratio (trailing), EPS (trailing). If possible from your source, add P/B or P/S.
def get_financial_ratios(ticker_s):
    ticker = yf.Ticker(ticker_s)
    info = ticker.info
    financials = ticker.financials.iloc[:,0]
    balance_sheet = ticker.balance_sheet.iloc[:,0]
    cashflow = ticker.cashflow.iloc[:,0]


    

    market_cap = info.get('marketCap')
    pe = info.get('trailingPE')
    forward_pe = info.get('forwardPE')
    eps = info.get('trailingEps')
    forward_eps = info.get('forwardEps')
    pb= info.get('priceToBook')

    ebit = financials.get('EBIT', 0)
    total_debt = balance_sheet.get('Total Debt', 0)
    total_equity = balance_sheet.get('Stockholders Equity', 0) 
    capital_employed = total_equity + total_debt

    if capital_employed != 0:
        roce = (ebit / capital_employed) * 100
    else:
        roce = None

    net_income = financials.get('Net Income', 0)
    if total_equity != 0:
        roe = (net_income / total_equity) * 100
    else:
        roe = None

    revenue = financials.get('Operating Revenue', financials.get('Total Revenue', 0))
    operating_income = financials.get('Operating Income', 0)

    if revenue != 0:
        operating_margin = (operating_income / revenue) * 100
        net_profit_margin = (net_income / revenue) * 100
    else:
        operating_margin = None
        net_profit_margin = None

    if market_cap is not None and revenue != 0:
        ps = market_cap / revenue
    else:
        ps = None


    if total_equity != 0:
        debt_to_equity = total_debt / total_equity
    else:
        debt_to_equity = None

    operating_cash = cashflow.get('Operating Cash Flow', 0)
    capex = cashflow.get('Capital Expenditures', 0)
    fcf = operating_cash - abs(capex)

    if net_income != 0:
        ocf = operating_cash / net_income
    else:
        ocf = None

    company_ratios = (
            f" Market Cap: {market_cap}\n"
            f" Trailing Price to Earning: {pe}\n"
            f" Forward Price to Earning: {forward_pe}\n"
            f" Trailing EPS: {eps}\n"
            f" Forward EPS: {forward_eps}\n"
            f" Price to Book: {pb}\n"
            f" Price to Sales: {ps}\n"
            f" ROCE: {roce}\n"
            f" ROE: {roe}\n"
            f" Operating Margin: {operating_margin}\n"
            f" Net Profit Margin: {net_profit_margin}\n"
            f" Debt to Equity: {debt_to_equity}\n"
            f" Free Cash Flow: {fcf}\n"
            f" Operating Cash Flow: {ocf}"
        )
    return company_ratios


#Fetches top headlines
def get_company_news(ticker_s):
    ticker = yf.Ticker(ticker_s)
    news = ticker.news

    if not news:
        return "No recent news found for this ticker."

    headlines_summary = "Recent News Headlines:\n"
    for i, article in enumerate(news[:5]):
        title = article['content'].get('title')
        summary = article['content'].get('summary')
        publisher = article['content'].get('publisher')
        headlines_summary += f"{i+1}. {title}, Summary: {summary}, Source: {publisher}\n"
    
    return headlines_summary.strip()




In [4]:
tool_get_company_info = Tool(
    name="get_company_info",
    func=get_company_info,
    description="Useful for fetching the company's name, sector, industry, a detailed business summary, and its current stock price. Input should be a stock ticker symbol."
)

tool_get_financial_ratios = Tool(
    name="get_financial_ratios",
    func=get_financial_ratios,
    description="Useful for fetching key financial ratios for a company, including Market Cap, P/E ratios, EPS, P/B, P/S, ROCE, ROE, profit margins, D/E ratio, and Free Cash Flow. Input should be a stock ticker symbol."
)


tool_get_company_news = Tool(
    name="get_company_news",
    func=get_company_news,
    description="Useful for fetching the most recent news headlines (typically 3) for a given stock ticker symbol to understand current events and sentiment. Input should be a stock ticker symbol."
)

tools = [tool_get_company_info, tool_get_financial_ratios, tool_get_company_news]

In [22]:
SYSTEM_PROMPT = """
You are a Basic Financial Analyst Assistant. Your goal is to provide a proper fundamental analysis of a company based on its stock ticker.
You MUST use the provided tools to gather all necessary information before formulating your analysis. 
DO NOT use any information outside of what the tools provide for the company's data.

Follow these steps METICULOUSLY:

When a user provides a ticker symbol:

1.  **Information Gathering Phase:**
    * Use the `GetCompanyProfileAndPrice` tool to get the company's profile and current price.
    * Use the `GetKeyFinancialRatios` tool to get its key financial metrics.
    * Use the `GetRecentCompanyNews` tool to get the latest news headlines.
    * Ensure you have gathered data from all three tools.

2.  **Analysis and Reporting Phase (Once all tool data is available):**
    * **Synthesize Information:**
        * Present a clear summary of the company profile: name, sector, industry, and a brief on what it does.
        * State the current stock price.
        * List the key financial ratios retrieved (P/E, EPS, etc.). Note if an industry benchmark P/E was provided (tool description says it's usually not available).
        * Summarize the recent news.
    * **Basic Valuation Analysis & "Take":**
        * Based SOLELY on the retrieved tool data and the following simple heuristics, provide a preliminary valuation take (potentially overvalued, undervalued, or fairly valued). Clearly state your reasoning.
        * **P/E Ratio Context:**
            * If an industry average P/E is available (unlikely with current tools): (heuristics as you defined)
            * If no industry average P/E is available: Comment on the P/E in general terms (e.g., "A P/E of [value] is generally considered high/moderate/low for many companies, but without an industry benchmark, it's hard to make a direct comparison."). Lean towards a valuation take if other factors (EPS, news) are strongly suggestive. For very high P/Es (> 50-60) without strong growth (from EPS) or positive news, lean towards *potentially overvalued*. For very low P/Es (< 10-12) with stable EPS and neutral/positive news, lean towards *potentially undervalued*.
        * **EPS (Earnings Per Share):** Positive and ideally growing EPS is good. Negative EPS is a concern. Relate to P/E.
        * **Recent News Sentiment:** Infer sentiment (positive, negative, neutral) from the news headlines and factor this into your reasoning.
    * **Construct Your Response:**
        * Combine the synthesis and your valuation take with clear reasoning into a final report format.
        * Start your response with "Basic Financial Analysis Report for [Ticker] as of [Current Date/Time]:". (You will need to get the current date/time when formatting the final input to this prompt or tell the agent to state it)
        * End with the disclaimer: "This is a basic analysis based on limited, real-time data and is not financial advice. Consult with a qualified financial advisor before making investment decisions."
    * **IMPORTANT:** If a tool returns an error or "N/A" for crucial data, acknowledge this limitation in your analysis (e.g., "P/E ratio was not available, limiting valuation assessment.").
"""



def financial_agent(llm_instance, tools, system_prompt):

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    
    agent = create_tool_calling_agent(llm_instance, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return agent_executor


if __name__ == "__main__":
    
    load_dotenv()
    google_api_key = os.getenv("GOOGLE_API_KEY")
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.0)

    financial_agent_res = financial_agent(llm, tools, SYSTEM_PROMPT)
    ticker = input("Enter the Ticker/Company You Wanna Analyze!")
    
    if ticker:
        current_time_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S %Z')
        agent_input = f"Generate a fundamental analysis report for the ticker: {ticker} and give your final conclusion of Fairly Valued, Over Valued or Under Valued. The current date and time for the report is: {current_time_str}."

        response = financial_agent_res.invoke({"input": agent_input})
        print("\n--- Agent's Final Output ---")
        print(response.get("output"))



> Entering new AgentExecutor chain...

Invoking: `get_company_info` with `HCLTECH.NS`


Company Name: HCL Technologies Limited
Sector: Technology
Industry: Information Technology Services
Current Market Price: 1659.90
Business Summary: HCL Technologies Limited provides IT and business services, engineering and research and development services, and modernized software products and IP-led offerings. It operates through IT and Business Services; Engineering and R&D Services; and HCL Software segments. The company provides application development, management, modernization, and testing, as well as commercial applications; automation services, including digital integration, business process management, robotic process automation for intelligent automation, and low and no code services; digital process operations, such as customer experience management, hyper intelligent automation, supply chain management, finance and accounting, marketing operations and content, and human resource servi

# Imp Functions:
Ticker: single ticker data <br>
Tickers: multiple tickers' data <br>
download: download market data for multiple tickers <br>
Market: get information about a market <br>
WebSocket and AsyncWebSocket: live streaming data <br>
Search: quotes and news from search <br>
Sector and Industry: sector and industry information <br>


# IMP Ratios:

### Business Quality:
1. ROCE
2. ROE
3. EPS
4. Operating Margins & Net Profit Margins

### Valutaion:
1. P/E
2. P/B
3. P/S

### Financial Health:
1. Debt-to-Equity

### Cash Flow:
1. Free Cash Flow
2. Operating Cash Flow/Net Profit

### Management:
1. High Promoter Holding & Low/Zero Pledging
